In [1]:
import pandas as pd
import os
import json
from pathlib import Path 

In [2]:
train_json_path = Path('../data/train.json')
val_json_path = Path('../data/val.json')

# train_data = pd.read_json(train_json_path)
train_json = json.load(train_json_path.open())
val_json = json.load(val_json_path.open())

# train_data.head()

In [3]:
# print(train_json['annotations'])
train_data_prearranged = pd.read_json(json.dumps(train_json['annotations']))
val_data_prearranged = pd.read_json(json.dumps(val_json['annotations']))

train_data_prearranged.head()

,area,iscrowd,id,image_id,category_id,bbox
0,2304645,0,1,10,4,"[704, 620, 1401, 1645]"
1,55769,0,2,1000,1,"[321, 332, 217, 257]"
2,17108,0,3,10003,2,"[220, 758, 188, 91]"
3,117909,0,4,10003,5,"[150, 397, 297, 397]"
4,105925,0,5,10003,1,"[85, 207, 475, 223]"


In [4]:
# Commented out for now. Will probably be used later in the test.
# train_data = pd.DataFrame(train_data_prearranged,columns=['image_id','category_id','bbox','area'])



In [5]:
# Use this DataFrame with flow_from_dataframe method of the ImageDataGenerator class to generate data set. Example method call below
# ImageDataGenerator.flow_from_dataframe(train_data_flowvers, directory='../data/train/train',class_mode = 'categorical', x_col = 'image_id', y_col = 'category_id',save_to_dir = '../data/train/results')
train_data_flowvers = pd.DataFrame(train_data_prearranged,columns=['image_id','category_id'])
val_data_flowvers = pd.DataFrame(val_data_prearranged,columns=['image_id','category_id'])

train_data_flowvers = train_data_flowvers.astype('str')+'.jpg'
val_data_flowvers = val_data_flowvers.astype('str') + '.jpg'
# val_data_flowvers.index
train_data_flowvers.head()

,image_id,category_id
0,10.jpg,4.jpg
1,1000.jpg,1.jpg
2,10003.jpg,2.jpg
3,10003.jpg,5.jpg
4,10003.jpg,1.jpg


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [7]:
num_classes = 5
# Downloaded from https://www.kaggle.com/keras/resnet50
# resnet_weights_path = '../data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

new_model = Sequential()
new_model.add(ResNet50(include_top=False,pooling = 'avg', weights = 'imagenet'))
new_model.add(Dense(num_classes,activation='sigmoid'))
# sigmoid, softmax

new_model.layers[0].trainable = False

In [8]:
new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_dataframe(train_data_flowvers, directory='../data/train/train',class_mode = 'categorical', x_col = 'image_id', y_col = 'category_id')

val_generator = data_generator.flow_from_dataframe(val_data_flowvers, directory='../data/val/val',class_mode = 'categorical', x_col = 'image_id', y_col = 'category_id')

history = new_model.fit(
        train_generator,
        steps_per_epoch=416,
        validation_data=val_generator,
        validation_steps=76)
new_model.save('saved_models/RESNET50_default_sigmoid')

Found 13317 validated image filenames belonging to 5 classes.
Found 2458 validated image filenames belonging to 5 classes.
416/416 [==============================] - 1235s 3s/step - loss: 1.2140 - accuracy: 0.5683 - val_loss: 1.2723 - val_accuracy: 0.5448
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_models/RESNET50_default_sigmoid/assets
